# A code to quickly check the *trajectory.nc* file content

Define here the name of the file to check

In [1]:
data_filename = '3_VarLight/2021-12-21/1/trajectory.nc'

In [8]:
from ipywidgets import interact, interactive
import utilities

import ipywidgets as widgets

from IPython.display import Markdown, display
from datetime import timedelta
import xarray as xr
import os, sys
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget


In [3]:
root = '/Users/baptistelafoux/Google Drive/Pro/2020_PMMH_thèse_fish/5_PostProcessing/postProCode/python/free_swim_illum_var/'
root_server = '/Volumes/baptiste/data_labox/illuminance_variation/1_raw_data/'

original_video_filename = glob.glob(root_server + os.path.dirname(data_filename) + '/*noBG.mp4')[0]

ds = xr.open_dataset(root + 'cleaned/' + data_filename)
original_video_cap = cv2.VideoCapture(original_video_filename)

print('Video file :', original_video_filename)
print('Data file :', data_filename)



Video file : /Volumes/baptiste/data_labox/illuminance_variation/1_raw_data/3_VarLight/2021-12-21/1/2021-12-21_noBG.mp4
Data file : 3_VarLight/2021-12-21/1/trajectory.nc


Metadatas

In [4]:
dt = 1 / ds.fps

display(Markdown('**Video**'))
print('Frame rate : {} fps'.format(ds.fps))
print('Video length : {} images ('.format(ds.n_frames) + str(timedelta(seconds=int(ds.n_frames/ds.fps))) + ')')

display(Markdown('**Content**'))
print('Number of individuals : {} fish'.format(ds.n_fish))
print('Average body length : {:.2f} pxl'.format(ds.mean_BL_pxl))

**Video**

Frame rate : 5 fps
Video length : 21002 images (1:10:00)


**Content**

Number of individuals : 58 fish
Average body length : 43.16 pxl


In [5]:
trail_len = 5
delta_t = 100

fig, ax = plt.subplots(1, figsize=(16, 5))

ax = [ax]

def display_movie(t=trail_len):

    ax[0].clear()

    original_video_cap.set(cv2.CAP_PROP_POS_FRAMES, t)
    _, frame = original_video_cap.read()

    ax[0].imshow(frame)
    ax[0].invert_yaxis()
    ax[0].plot(ds.s[t-trail_len:t, :, 0] * ds.mean_BL_pxl,
              ds.s[t-trail_len:t, :, 1] * ds.mean_BL_pxl, linewidth=0.75)
    ax[0].plot(ds.s[t, :, 0] * ds.mean_BL_pxl, ds.s[t, :, 1] * ds.mean_BL_pxl, 'k.')

    #for focal in range(ds.n_fish):
    #   ax[0].annotate(str(focal), (ds.s[t, focal, 0], ds.s[t, focal, 1]), xytext=(2, 5), textcoords='offset points')

    #ax[0].axis('scaled')
    #ax[0].axis([0, ds.tank_size[0], 0, ds.tank_size[1]])

        
slider = interactive(display_movie, t=widgets.IntSlider(min=101, max=ds.n_frames-1, step=1))

display(slider)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=101, description='t', max=21001, min=101), Output()), _dom_classes=('wid…

In [56]:
r = ds.s - ds.center_of_mass
v = ds.v
N = ds.n_fish

R_max = 20
i = ds.ii_dist.mean(dim='neighbour') < R_max

print(np.sum(i) / 58)

print(i.shape) 
print(r.shape)
r *= i
v *= i

plt.figure()
plt.plot(ds.time, np.min(ds.ii_dist.mean(dim=['neighbour', 'fish']))/ds.ii_dist.mean(dim=['neighbour', 'fish']))

plt.figure()
plt.plot(ds.time, np.linalg.norm(r, axis=-1))
print(r.shape)

# we add an epilon to the norm of the velocity in case is it 0
rotation_parameter = np.sum(np.cross(r, v) / (np.linalg.norm(r, axis=2) * (np.linalg.norm(v, axis=2) + 10**(-9))), axis=1) / N

plt.figure()
plt.plot(ds.time, rotation_parameter)


<xarray.DataArray 'ii_dist' ()>
array(19714.)
(21002, 58)
(21002, 58, 2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(21002, 58, 2)


/var/folders/6g/_pvh03v53030m1dmqwzw8b0r0000gn/T/ipykernel_19694/1406645278.py:23: RuntimeWarning: invalid value encountered in true_divide
  rotation_parameter = np.sum(np.cross(r, v) / (np.linalg.norm(r, axis=2) * (np.linalg.norm(v, axis=2) + 10**(-9))), axis=1) / N


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

13.323806682087389